In [1]:
import pandas as pd
import lightgbm as lgb
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# --- 1. Chargement et Préparation des Données ---
print("--- Étape 1 : Entraînement du Modèle de Risque (Le Vigile) ---")
input_path = "../../data/processed/dataset_clean_no_outliers.parquet"
output_dir = "../../models/"
os.makedirs(output_dir, exist_ok=True)

df = pd.read_parquet(input_path)

# Création de la variable cible binaire
df['is_default'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

# --- 2. Sélection des Variables Pertinentes ---
# On utilise la liste de "championnes" validée lors de l'analyse préliminaire
selected_features_for_risk_model = [
    'term', 'annual_inc', 'dti', 'revol_util', 'revol_bal', 'loan_amnt', 
    'emp_length', 'home_ownership', 'purpose', 'verification_status', 
    'mort_acc', 'pub_rec', 'open_acc', 'total_acc'
]

X = df[selected_features_for_risk_model]
y = df['is_default']

# Encodage des variables catégorielles
X = pd.get_dummies(X, drop_first=True, dtype=float)

# --- 3. Division des Données ---
# On utilise 'stratify' car les classes (défaut/non-défaut) sont déséquilibrées
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 4. Entraînement du Modèle LightGBM ---
print("\nEntraînement du LGBMClassifier...")

# Calcul du poids pour gérer le déséquilibre des classes
ratio = y_train.value_counts()[0] / y_train.value_counts()[1]

model_1 = lgb.LGBMClassifier(
    objective='binary',
    metric='auc',
    random_state=42,
    scale_pos_weight=ratio
)

model_1.fit(X_train, y_train)
print("✅ Modèle de risque entraîné.")

# --- 5. Évaluation du Modèle ---
y_pred_proba = model_1.predict_proba(X_test)[:, 1]
y_pred_class = model_1.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"\n--- Performance du Modèle 1 ---")
print(f"Score AUC : {auc_score:.4f}")
print("\n--- Rapport de Classification ---")
print(classification_report(y_test, y_pred_class))
print("\n--- Matrice de Confusion ---")
print(confusion_matrix(y_test, y_pred_class))

# --- 6. Sauvegarde des Artefacts ---
# On sauvegarde le modèle et la liste exacte de ses colonnes
model_1_path = os.path.join(output_dir, "step1_risk_model.pkl")
model_1_cols_path = os.path.join(output_dir, "step1_risk_model_columns.pkl")

joblib.dump(model_1, model_1_path)
joblib.dump(X_train.columns.tolist(), model_1_cols_path)

print(f"\n✅ Modèle 1 sauvegardé dans : {model_1_path}")
print(f"✅ Colonnes du Modèle 1 sauvegardées dans : {model_1_cols_path}")

--- Étape 1 : Entraînement du Modèle de Risque (Le Vigile) ---

Entraînement du LGBMClassifier...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 56440, number of negative: 236531
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 292971, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.192647 -> initscore=-1.432901
[LightGBM] [Info] Start training from score -1.432901
✅ Modèle de risque entraîné.

--- Performance du Modèle 1 ---
Score AUC : 0.6944

--- Rapport de Classification ---
              precision    recall  f1-score   support

           0       0.88      0.64      0.74     59133
           1       0.30      0.64    